In [1]:
# import system libs
# import os
# import cv2
# import numpy as np
# import pandas as pd
# import seaborn as sns
# import random
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix, classification_report
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
# from tensorflow.keras.utils import image_dataset_from_directory

import os
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from keras.src.legacy.preprocessing.image import ImageDataGenerator
from keras.src.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from keras.src.utils import image_dataset_from_directory
from PIL import Image



In [2]:
data_dir = "C:\\Users\\hp\\PycharmProjects\\Retina_Optik_Disc\\retina_optic_disc"

In [3]:
train_dir = os.path.join(data_dir  + "\\train")
print("Train Directory --> ", os.listdir(train_dir))

validation_dir = os.path.join(data_dir  + "\\val")
print("Validation Directory --> ", os.listdir(validation_dir))

test_dir = os.path.join(data_dir  + "\\test")

print("Test Directory --> ", os.listdir(test_dir))

Train Directory -->  ['healty', 'unhealty']
Validation Directory -->  ['healty', 'unhealty']
Test Directory -->  ['healty', 'unhealty']


In [5]:
# data_dir = "C:\\Users\\hp\\PycharmProjects\\Retina_Optik_Disc\\retina_optic_disc"

# def create_dataframe(data_path):
#     filepaths = [] 
#     labels = [] 

#     # Ana klasördeki alt klasörleri (folds) listeleme
#     folds = os.listdir(data_path)
#     for fold in folds:
#         # Alt klasördeki görüntülerin yollarını almak için yol oluşturma
#         f_path = os.path.join(data_path, fold)
#         imgs = os.listdir(f_path)  # Alt klasördeki görüntüleri listeleme

#         # Her görüntü için dosya yolunu ve sınıf etiketini ekleme
#         for img in imgs:
#             img_path = os.path.join(f_path, img)  # Görselin tam dosya yolunu oluşturma
#             filepaths.append(img_path)  # Görselin yolunu listeye ekleme
#             labels.append(fold)         # Görselin etiketini listeye ekleme
    
#     # Dosya yolları ve etiketlerin pandas Series'e dönüştürülmesi
#     fseries = pd.Series(filepaths, name='Filepaths')
#     lseries = pd.Series(labels, name='Labels')

#     # İki seriyi birleştirip bir DataFrame olarak döndürme
#     return pd.concat([fseries, lseries], axis=1)

# retina_df = create_dataframe(data_dir)

In [6]:
# def prepare_image(folder, target=(124,124), quality=85):
#     for file in os.listdir(folder):
#         if file.endswith(".jpeg"):
#             image_path = os.path.join(folder, file)
#             with Image.open(image_path) as image:
#                 resized_image = image.resize(target)
#                 # Sıkıştırma seviyesini 85 olarak belirleyip kaydet
#                 resized_image.save(image_path, quality=quality)

# folder = "C:\\Users\\hp\\PycharmProjects\\Retina_Optik_Disc\\retina_optic_disc"
# prepare_image(folder)

In [4]:
def create_dataframe(data_path):
    filepaths = []
    labels = []

    # Ana klasördeki alt klasörleri (folds) listeleme
    folds = os.listdir(data_path)
    for fold in folds:
        # Alt klasördeki görüntülerin yollarını almak için yol oluşturma
        f_path = os.path.join(data_path, fold)
        imgs = os.listdir(f_path)  # Alt klasördeki görüntüleri listeleme

        # Her görüntü için dosya yolunu ve sınıf etiketini ekleme
        for img in imgs:
            img_path = os.path.join(f_path, img)  # Görselin tam dosya yolunu oluşturma
            filepaths.append(img_path)  # Görselin yolunu listeye ekleme
            labels.append(fold)  # Görselin etiketini listeye ekleme

    # Dosya yolları ve etiketlerin pandas Series'e dönüştürülmesi
    fseries = pd.Series(filepaths, name='Filepaths')
    lseries = pd.Series(labels, name='Labels')

    # İki seriyi birleştirip bir DataFrame olarak döndürme
    return pd.concat([fseries, lseries], axis=1)


train_retina_df = create_dataframe(train_dir)
test_retina_df = create_dataframe(test_dir)
validation_retina_df = create_dataframe(validation_dir)

print(train_retina_df)
print(test_retina_df)
print(validation_retina_df)

                                              Filepaths    Labels
0     C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...    healty
1     C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...    healty
2     C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...    healty
3     C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...    healty
4     C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...    healty
...                                                 ...       ...
5354  C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...  unhealty
5355  C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...  unhealty
5356  C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...  unhealty
5357  C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...  unhealty
5358  C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...  unhealty

[5359 rows x 2 columns]
                                              Filepaths    Labels
0     C:\Users\hp\PycharmProjects\Retina_Optik_Disc\...    healty
1     C:\Users\hp\PycharmProjects\Retina_Optik_Disc